# Lab Report #4

We were tasked with investigating the dynamics of a mass-spring system under the influence of multiple forces. The main focus was to understand the role of parallel and perpendicular force components in shaping the motion of the system. Additionally, we extended our simulation to study the effects of damping and to interpret the behavior of kinetic energy throughout the motion.

## Problem Statement
**Question**: What are the two vectors (parallel and perpendicular components of the net force) doing to the motion, and how do we see that in the simulation? How are these two components of net force affecting our motion?

To answer this, we aimed to:
1. Break down the net force acting on the mass into its parallel and perpendicular components.
2. Examine how each component influences the trajectory and velocity of the ball.
3. Incorporate damping effects to analyze how energy dissipates over time.
4. Investigate how kinetic energy evolves and confirm whether our results align with physical expectations.

## Physics Relevant to Our Problem
Several physics concepts and formulas guided the simulation:
* Spring Force:
    - Formula: $F=−kx$
    - Description: The restoring force from the spring, proportional to displacement, acting in the opposite direction.
* Gravitational Force:
    - Formula: $F=mg$
    - Description: The constant downward force on the ball, only in the y-direction.
* Drag Force (Damping):
    - Formula: $F_d=−μ∣v∣$
    - Description: The velocity-dependent damping force, acting opposite to motion, scaling with the damping coefficient.
* Net Force:
    - Description: A combination of all forces: (1) spring, (2) gravity, and (3) damping. This total force determines acceleration at each step.

* Momentum Principle:
    - Formula: $\vec{p}_f=\vec{p}_i+\vec{F}_{net}Δt$
    - Description: Updates momentum and, through $\vec{v}=\frac{\vec{p}}{m}$, velocity. Position updates via $\vec{r}_f=\vec{r}_i+\vec{v}Δt$.
* Parallel and Perpendicular Force Components:
    - Parallel Force:
        - Formula: $\vec{F}=\hat{p}(\vec{F} \times \hat{p})$
        - Description: Represents the portion of net force aligned with the object’s motion, changing its speed (magnitude of momentum).
    - Perpendicular Force:
        - Formula: $\vec{F}_\perp=\vec{F}-\vec{F}_\parallel$
        - Description: Represents the portion of net force orthogonal to the motion, changing the direction of momentum.
* Kinetic Energy:
    - Formula: $KE=\frac{1}{2} \times mv^2$
    - Description: Scalar quantity reflecting the ball’s energy of motion. Used to monitor energy transfer and the effects of damping.

##  How the Code Solves Our Problem

To build on the previous lab, we implemented new code that visualizes the forces acting on the ball as arrows. This allows us to not only compute the physics numerically but also see the relative magnitudes and directions of the forces at play.
* Initialization of Force Attributes:
    * Attributes were created for the ball object to store gravity, spring force, drag force, net force, and the parallel and perpendicular components.
* Arrow Visualization:
    * Arrows were attached to the ball to represent each force vector and the momentum vector. These provide a dynamic visual reference for how the forces evolve as the simulation runs.
* Updating Within the While Loop:
    * The same code segments were included inside the loop to update the vectors each timestep. This ensures the arrows reflect the ball’s current state of motion.
* Kinetic Energy:
    * Kinetic energy was implemented as $KE=\frac{1}{2}mv^2$. The value was stored as an attribute of the ball and plotted over time.

## The Code

In [1]:
from vpython import *

class Dummy():
    pass

def run_simulation(show_simulation=True, velocity_graph=False, position_graph=False, work_graph=False,
                   kinetic_energy_graph=False, print_message=False):
    """
    Run a simplified mass–spring system simulation and optionally display 
    VPython graphs.
    
    Args:
        show_simulation (bool):
            If True, creates VPython 3D objects (Sun and Earth) and shows their 
            orbital motion.

        velocity_graph (bool):
            If True, plots the velocity of the ball.

        position_graph (bool):
            If True, plots the position of the ball in (x, y, z) plane.

        work_graph (bool):
            If True, plots the work done by the ball.

        kinetic_energy_graph (bool):
            If True, plots the kinetic energy of the ball.
            
        print_message (bool):
            If True, prints the measured and theoretical periods associated 
            with the spring and ball.
    """
    
    # Constants and parameters
    l_0 = 6                             # initial length of the spring
    alpha = 40 * (pi/180)               # initial angle
    l_0_spring = 2 * l_0                # stretched spring length
    k_spring = 10                       # spring constant k
    g = 10                              # gravitational acceleration
    mu = 0.02                           # damping coefficient

    if show_simulation:
        # Fixed point at the ceiling where spring is attached
        fix_point = sphere(pos = vec(0, 6, 0),
                           radius = 0.5,
                           color = color.red)
        
        # Setup for mass object
        ball = sphere(pos = fix_point.pos + vec(l_0_spring * sin(alpha), -l_0_spring * cos(alpha), 0),
                      mass = 0.5,
                      v = vec(0, 0, 0),                                   # initial velocity
                      radius = 0.1, color = color.blue,
                      make_trail = True,
                      trail_type = "points",
                      interval = 10,
                      retain = 100,
                      trail_radius = 0.05)
        
        # Setup for spring object
        spring = helix(pos = fix_point.pos,
                       axis = ball.pos - fix_point.pos,
                       radius = 0.2,
                       color = color.orange)
    else:
        fix_point = Dummy()
        ball = Dummy()
        spring = Dummy()

        fix_point.pos = vec(0, 6, 0)
        ball.pos = fix_point.pos + vec(l_0_spring * sin(alpha), -l_0_spring * cos(alpha), 0)
        ball.mass = 0.5
        ball.v = vec(0, 0, 0)
        spring.pos = fix_point.pos
    
    # Initial momentum of ball
    ball.p = ball.mass * ball.v
    
    # Initial force calculations
    l = ball.pos - fix_point.pos
    spring.s = mag(l) - l_0  # displacement between the spring's initial position and its final position
    
    # Simulation parameters
    my_rate = 160
    t = 0
    dt = 0.005
    
    
    # Forces
    F_s = - k_spring * spring.s * l.hat     # spring
    F_g = ball.mass * vec(0, -g, 0)         # gravity
    F_d = - mu * ball.v.mag * ball.v.hat    # damping
    F = F_g + F_s + F_d                     # net
    
    # Store forces in ball object for visualization
    ball.fg = F_g
    ball.fs = F_s
    ball.fd = F_d
    ball.f = F
    
    # Force components parallel/perpendicular to momentum
    ball.f_parallel = ball.p.hat * (ball.f.dot(ball.p.hat) )
    ball.f_perpendicular = ball.f - ball.f_parallel

    # Force arrows for visual aid
    if show_simulation:
        myarrowfg = attach_arrow(ball, "fg", scale=0.2, shaftwidth=0.1, color=color.green)
        myarrowfs = attach_arrow(ball, "fs", scale=0.2, shaftwidth=0.1, color=color.purple)
        myarrowfd = attach_arrow(ball, "fd", scale=0.2, shaftwidth=0.1, color=color.cyan)
        myarrowf = attach_arrow(ball, "f", scale=0.2, color=color.magenta)
        myarrowp = attach_arrow(ball, "p", scale=0.2, color=color.orange)
        myarrowparallel = attach_arrow(ball, "f_parallel", scale=0.2, color=color.red)
        myarrowperpendicular = attach_arrow(ball, "f_perpendicular", scale=0.2, color=color.yellow)
    
    # Ball velocity from momentum
    ball.v = ball.p / ball.mass
    
    # Initial energy values
    ke = 1/2*ball.mass * ball.v.mag2
    ke_0 = 1/2*ball.mass * mag(ball.v)**2 
    work = 0
    work = work + ball.f.dot(ball.v) * dt # deviation is a part of computational error
    ball.ke = ke

    # Graph velocity components if required
    if velocity_graph:
        v_graph = graph(title = ' Velocity ', xtitle = ' t ', ytitle = ' v(t) ')
        vx_b = gcurve(graph = v_graph, color = color.cyan)
        vy_b = gcurve(graph = v_graph, color = color.magenta)
        vz_b = gcurve(graph = v_graph, color = color.blue)
    
    # Graph position components if required
    if position_graph:
        pos_graph = graph(title = ' Position ', xtitle = ' t ', ytitle = ' x(t) ')
        pos_x = gcurve(graph = pos_graph, color = color.yellow)
        pos_y = gcurve(graph = pos_graph, color = color.orange)
        pos_z = gcurve(graph = pos_graph, color = color.red)
    
    # Graph work components if required
    if work_graph:
        w_graph = graph(title = ' Work ', xtitle = ' t ', ytitle = ' Work ')
        work_plot = gcurve(graph = w_graph, color = color.green)
    
    # Graph kinetic energy components if required
    if kinetic_energy_graph:
        kinetic_graph = graph(title = ' Kinetic Energy ', xtitle = ' t ', ytitle = ' KE ')
        ke_scalar = gcurve(graph = kinetic_graph, color = color.red)
    
    # For period measurement
    ball.T = 0
    ball.old_v = ball.v
    
    period_num = 1
    
    # Simulation Loop (10 seconds)
    while t < 10:
        rate(my_rate)
        
        # Update spring axis
        spring.axis = ball.pos - fix_point.pos
        l = ball.pos - fix_point.pos
        spring.s = mag(l) - l_0
        
        # Recalculate forces
        F_s = - k_spring * spring.s * l.hat     # spring in l^hat direction
        F_g = ball.mass * vec(0, -g, 0)         # gravitational 
        #F_d = -mu * ball.v.mag2 * ball.v.hat   # quadratic
        F_d = - mu * ball.v.mag * ball.v.hat    # linear
        F = F_g + F_s + F_d                     # total
        
        # Store forces in ball object
        ball.fg = F_g
        ball.fs = F_s
        ball.fd = F_d
        ball.f = F
        
        # Parallel and perpendicular force components relative to momentum
        ball.f_parallel = ball.p.hat * (ball.f.dot(ball.p.hat) )
        ball.f_perpendicular = ball.f - ball.f_parallel
        
        # Momentum and velocity updates
        ball.p = ball.p + F * dt
        ball.v = ball.p / ball.mass
        ball.pos = ball.pos + ball.v * dt
        
        # Kinetic energy
        ke = 1/2 * ball.mass * mag(ball.v)**2
        work = work + ball.f.dot(ball.v) * dt   # deviation is a part of computational error
        ball.ke = ke
        
        # Plot velocity graph
        if velocity_graph:
            vx_b.plot(pos = (t, ball.v.x))
            vy_b.plot(pos = (t, ball.v.y))
            vz_b.plot(pos = (t, ball.v.z))
        
        # Plot position graph
        if position_graph:
            pos_x.plot(pos = (t, ball.pos.x))
            pos_y.plot(pos = (t, ball.pos.y))
            pos_z.plot(pos = (t, ball.pos.z))

        if work_graph:
            work_plot.plot(pos = (t, work))
        
        # Plot work and kinetic energy graphs
        if kinetic_energy_graph:
            #ke_scalar.plot(pos = (t, ball.ke))
            ke_scalar.plot(pos = (t, ke - ke_0))
        
        # Increment time
        t += dt
        
        # Detect when velocity crosses from positive to negative (peak of oscillation)
        if print_message:
            if ball.old_v.y > 0 and ball.v.y < 0 :
                measured_period = round(ball.T, 2)
                theoretical_period = round(2 * pi * sqrt(ball.mass / k_spring), 2)
                
                print(f"Period #{period_num}\n\tMeasured Period: {measured_period} seconds\n\tTheoretical Period: {theoretical_period} seconds")
                ball.T = 0
                period_num += 1
            
            
        ball.old_v = ball.v
        ball.T = ball.T + dt


/home/gillty/Physic_Labs/jupyter_env/lib/python3.12/site-packages/vpython/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


<IPython.core.display.Javascript object>

## Results and Discussion
### Simulation Model
![Simulation](assets/gifs/Spring_Mass.gif)

In [2]:
simulation = canvas(background=color.white)

run_simulation(print_message=True)

<IPython.core.display.Javascript object>

Period #1
	Measured Period: 0.65 seconds
	Theoretical Period: 1.4 seconds
Period #2
	Measured Period: 1.14 seconds
	Theoretical Period: 1.4 seconds
Period #3
	Measured Period: 1.25 seconds
	Theoretical Period: 1.4 seconds
Period #4
	Measured Period: 1.33 seconds
	Theoretical Period: 1.4 seconds
Period #5
	Measured Period: 1.16 seconds
	Theoretical Period: 1.4 seconds
Period #6
	Measured Period: 1.29 seconds
	Theoretical Period: 1.4 seconds
Period #7
	Measured Period: 1.38 seconds
	Theoretical Period: 1.4 seconds
Period #8
	Measured Period: 1.32 seconds
	Theoretical Period: 1.4 seconds


&emsp;&emsp;This simulation was obtained by incorporating a damping coefficient of 0.02 while keeping the variables unchanged for the spring constant ($k = 10$), initial length of the spring ($l_0 = 6$), initial angle of the ball ($alpha = 40°$), the mass of the ball ($mass = 0.5$). However, it should be noted that the variable for ```l_0_spring``` was also modified to be 2 instead of 1.5, and as such we should expect the range of values for the graph to increase and the graph to stretch out vertically. As with the previous lab, this simulation showcases the movements of the ball as the spring causes it to constantly shift left and right in the x-direction while also shifting up and down in the y-direction. However, unlike the previous lab, we now have 5 arrows representative of the vectors for force (green, purple, cyan, magenta, red, and yellow)  and momentum (orange).

&emsp;&emsp;By the end of the simulation, the ball was moving in the north-west direction as the ball is in the process of retracting towards the fixed sphere as a result of the spring’s storing force. As is displayed, the purple vector representative for the force of the spring is the longest and pointing in the same direction that the ball is moving towards. This makes sense since the direction in which the restoring force of the spring is acting is in the same direction in which the ball is being pulled towards. It’s essentially being pulled toward equilibrium, and the momentum built up from that exceeds the other forces against this action. As for its length, the magnitude of the spring force should be anticipated to be the longest vector since the greatest force acting on the ball at this moment is the spring given that the ball is moving retracting back towards the base of the spring.

&emsp;&emsp;Meanwhile, the force of gravity shown in green is displayed moving downwards. This was expected to be the case due to the fact that gravity pulls objects towards the center of said gravity. Thus, the gravity vector would point directly down to the ground in the direction of the Earth’s core where its mass is centralized. Additionally, throughout the simulation, this vector did not change whatsoever despite the ball’s change in position and velocity. This makes sense since gravity is always constant at approximately $9.81 m/s^2$ on Earth. But for the sake of our simulation, we left it at $10 m/s^2$. Furthermore the Force of Gravity ($F_g = m \times g$) is also in terms of another constant that does not change throughout our problem which is the mass of the ball, which remains constant at a mass of 0.5 kg.

&emsp;&emsp;Moving on, the cyan vector representative of the force of drag is not visible in our initial diagram. But, we have scaled it up here as reference. But in terms of our initial diagram, this implies that the vector is so miniscule that it is not visible in the simulation. This was to be expected given that the variable mu involved in drag force is extremely small at 0.02. However, if we were to magnify the ball, like we did above, we would expect the vector for drag force to be moving in the opposite direction in which the ball is moving in. This is the case as it’s pointing back toward the path already traveled, which implies that it opposes the motion. This is because drag force is the opposing force acting against a system (in this case, our ball) as it moves through a medium of some sort. For this simulation, the dragging force acting on the ball is the result of air molecules in the pathway of the ball as it is in motion – thus slowing it down. This implies vector addition, in which two vectors going in opposite directions (positive and negative) are added to one another to create a new vector going in the direction of the larger of the two vectors. This would apply to this simulation with the Net Force vector (sum of all forces) shown in magenta being impacted by the opposing drag force vector. Thus, despite the fact that the drag force vector itself is not visible in our first diagram, its effects are as shown when we amplify the drag force (as shown in the picture above) with the reduced range at which the ball can travel from left to right as the drag on the ball slows it down and prevents it from being able to return to previously attainable positions.

&emsp;&emsp;The net force vector in magenta is shown moving in the north-west direction in such a way that it is below the purple vector symbolic of the force of the spring but higher than the green vector symbolic of the force of gravity. Additionally, the length of this particular vector is in between that of the purple and green vectors – being longer than the green vector but shorter than the purple vector. From a physicist’s  point of view, this would make sense given that the formula for net force is the sum of smaller force vectors. And taking into account that the addition of smaller vectors (force of gravity, force of spring, and force of drag) by way of overlapping the head of one vector on the tail of another would culminate in the creation of a larger resultant vector (our net force vector). Given that the force for drag is a negligible factor due to being so insignificantly short that it isn’t visible, we can ignore its involvement for the sake of understanding the net force in relation to our simulation. Thus, this leaves us with the force vectors for gravity and spring. With this in mind, we can confirm the legitimacy of our magenta force vector by visualizing the tail of the green vector being shifted over to overlap on the head of the purple vector. Doing this results in our magenta vector forming a bridge between the gravity and spring force vectors – connecting the head and tail respectively – with the net force vector fitting in nicely to create a triangle.
Moving on to the momentum vector fashioned in orange, we can see that by the end of the simulation as the ball is retracting towards the fixed sphere this vector is pointing in the same direction as the purple vector for spring force while possessing a smaller magnitude than it. To one well versed with the laws and rules associated with physics, this should make sense due to the fact that the direction of momentum is derived from velocity since the only vector involved in calculating momentum is such. At the moment of the screenshot in question, the ball is moving towards the fixed sphere and thus the velocity would also be going in that direction since velocity is simply the speed of an object in a particular direction.

&emsp;&emsp;Next, we have our perpendicular force vector in yellow and parallel force vector in red which are displayed in the screenshot pointing in the south-west and north-west direction. This makes sense given that the perpendicular force vector is – as its name implies – always perpendicular with the momentum as we observed in our simulation. Additionally, it’s important seeing as how the parallel force vector is pointing in the north-west direction, one can presume that the parallel force vector would be in the south-west direction and vice-versa. And it can also be inferred that the parallel force vector will be parallel to the momentum, as shown, as it seems that it remains on the same axis plane. As with net force and its respective smaller force vectors (drag force, force of gravity, and force of spring), the perpendicular and parallel force vectors along with net force form a triangle when their heads are overlapped with their tails. This shows that the net force is the resultant force vector for these two forces. Thus, in the grand scheme of things, the parallel and perpendicular force vectors help shape how the simulation of the ball will run. This is because as one force changes, the other changes in tandem with it so as to maintain a 90 degree angle constantly between them. Additionally, this impacts motion directly with the momentum, as shown in orange. The perpendicular force seems to change its direction and ultimate path of travel while the parallel force will control its speed in the same axis plane. So the parallel force will control how fast it's moving in a certain direction, while the perpendicular force will adjust that direction by turning a specific amount, thus leading to a new direction, and change in path of travel.

&emsp;&emsp;At the moment at which this screenshot was taken, the perpendicular force was quite short in comparison to the parallel force. Such a result does not require the power of foresight nor the use of a magic ball, but rather a good understanding of the physical properties of the universe. To put it simply, the perpendicular force is short due to the fact that it’s not going to change the direction, or the ball’s path of travel. However, it will grow to alter the path of travel, and the direction it’s going eventually. Meanwhile, the parallel force is much longer and stronger due to the fact that it’s causing the object to speed up. This is because the parallel force is regulating the speed at which the object is going in this moment. It can cause the Momentum to decrease, or increase, while the Perpendicular force causes a change in the direction. Below is a good diagram showcasing the Perpendicular Force in relation to the Momentum, about to change the direction of the system, and alter its path of travel. And when combined through vector addition, these two vectors create the resultant vector – net force.

## Graphs

### Graph #1 - Velocity

In [3]:
graph_1 = canvas()

run_simulation(show_simulation=False, velocity_graph=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;The graph shown above portrays the behavior of the ball’s velocity in the x, y, and z-directions. Particularly, what we are seeing is a constant fluctuation in the $x$ and y-directions between increasing and decreasing as well as dipping above and below the x-axis. Meanwhile, the blue line representative of the z-direction doesn’t change at all.

&emsp;&emsp;To start, the line for the z-component of velocity makes sense and behaves as expected since the program is designed to simulate the ball’s velocity in the $x$ and y-directions only. This is made evident by the fact that the z-component is set equal to 0 m/s for all vectors present in the formulas relevant to velocity such as the positions of the ball and fix_point, initial velocity of the ball, and our force vectors. As a result, whenever velocity is updated, the z-component is left untouched and thus stays constant at 0. Moving on to the y-component shown in magenta, we see a consistent trend of increasing and decreasing – both below and above the x-axis. This behaves as one should expect given that the ball is shown in the simulation to be constantly switching from moving upwards and downwards due to the physical conflict over equilibrium between the spring’s restoring force and the pull of gravity. With this in mind, one could predict that the graph would be similar to what is displayed above since the ball moving up is associated with moving in the positive direction while the ball moving down is associated with moving in the negative direction. 

&emsp;&emsp;Likewise, it should be expected for the ball to transition from increasing and decreasing due to the fact that the velocity in the y-direction will be increasing as the restoring force presented by the spring temporarily wins the battle over gravity and pulls it upwards to achieve equilibrium and decreasing whenever the ball slows down as it approaches its maximum or minimum y-value which presents the turning point at which both forces, the force of gravity, and the restoring force of the spring, become too great and overcome the other (drag force) to move it in the opposite direction. The spring force is always striving toward equilibrium, but it surpasses it in our simulation when moving up and down.

&emsp;&emsp;Additionally, the velocity increasing, and decreasing in the $y$ and $x$ directions reinforces that the acceleration is increasing, and slowing down at points within different directions as well. The velocity responds by speeding up due to an increase in acceleration, and slows down due to a decrease in acceleration, which is directly related to our Net Force. Carrying on to the x-component displayed in cyan, we see it initially start at 0 and move below the x-axis and follow a pattern of switching between decreasing and increasing wherein the ball would abruptly increase then decrease (or vice versa) in the span of a single second. Such behavior can be foreseen by one well versed in physics due to the fact that the acceleration from the summed Forces is initially increasing, and then decreases right after to achieve a velocity of 0 m/s. Then in the y, it initially speeds up in the opposite direction due to the Force being applied negatively with respect to the summed forces.

&emsp;&emsp;We chose this graph due to the fact that it’s interrelated with our parallel, and perpendicular forces to the Momentum. This is because the variable that is directly changing the Momentum is the velocity itself, which is represented by the equation $p = mv$. For this instance we can see that the parallel force is regulating the speed of the velocity, as it’s causing it to speed up, and slow down. However, because the perpendicular force is applying perpendicular to the velocity, it causes the direction to change, which is when the velocity reaches 0 m/s, as it’s forcing it to come to a stop before altering the direction entirely. So, we chose to use this velocity graph as one of our graphs due to the fact that it illustrates our motion in relation to the parallel and perpendicular force.


### Graph #3 - Position

In [4]:
graph_3 = canvas()

run_simulation(show_simulation=False, position_graph=True)

<IPython.core.display.Javascript object>

N/A

### Graph #4 - Work

In [5]:
graph_4 = canvas()

run_simulation(show_simulation=False, work_graph=True)

<IPython.core.display.Javascript object>

N/A

### Graph #5 - Kinetic Energy

In [6]:
graph_5 = canvas()

run_simulation(show_simulation=False, kinetic_energy_graph=True)

<IPython.core.display.Javascript object>

&emsp;&emsp;As shown above, the kinetic energy of the ball increases and decreases repeatedly above the x-axis. Moreover, as time progresses, the range of potential values for kinetic energy decreases. This can be explained by taking a closer look at the formula for kinetic energy, $KE = \frac {1}{2}mv^2$. Obviously, the variable for mass will remain constant given that no mass is added or removed throughout the simulation. Therefore, this means that the changes and fluctuations visible in the graph are attributable to velocity, $v$. This behaves as one would expect since the formula for velocity, $v = \frac{p}{m}$, is impacted by the formula for momentum, $p = FΔt$, which in turn is impacted by the formula for net force. To describe the broader significance of this, the drag force is a negative value which is included in the net force and acts as a resisting force against the ball’s movements. This, along with the resisting force from the pull of gravity in the y-direction aid in decreasing the velocity of the ball – therefore decreasing how much kinetic energy the ball is capable of building up over time.

&emsp;&emsp;Additionally, because the Kinetic Energy incorporates the $x$, $y$, and $z$ velocities respectively, as it’s a scalar & a magnitude, it shows that gradually as time moves forward, the Kinetic Energy decreases over time. This makes sense based on the velocity graph, as the velocity of the $x$ component gradually gets smaller over time to a noticeable amount to where the it makes sense that if we sum the $y$ and $x$ velocities, the magnitude of the velocities, as used in the Kinetic Energy equation, would gradually smaller over time. And because the Kinetic Energy is directly proportional to the magnitude of velocity squared, it makes sense that the Kinetic Energy would decrease over time, as shown in the graph, based on the magnitude of velocity decreasing as well due to the direct square proportion. Also, because the damping coefficient opposes the direction of motion, it causes the summed forces to oppose motion further as well, which would decrease the velocity. This also means that the Kinetic Energy would decrease further over time due to an increased damping coefficient, as it would resist the motion, and decrease the velocity. As previously mentioned, the velocity and Kinetic Energy have a square proportion, so if you decrease the velocity over time, it will decrease the Kinetic Energy as well.

&emsp;&emsp;We chose this graph since it depicts the motion in terms of the speed traveled. This shows that it’s slowing down over time in all directions. And it also displays the relation of the damping coefficient with respect to the velocity and Kinetic Energy alike. The damping coefficient is essentially causing the Net Force to oppose motion, which lowers the velocity, and in turn, it lowers the Kinetic Energy as a result. We chose this graph because the Kinetic Energy shows how the magnitude of the velocity decreases over time.


## Conclusion

**Summary of the Lab**:
In this lab, we extended the code from our previous assignment by implementing 3D vectors to represent the forces acting on the ball. We ran a single simulation that produced two graphs: one for velocity components and another for kinetic energy.

**Findings**:
* Parallel Force Component: Regulates the speed and momentum of the ball. It causes the object to speed up when increased, slow down when decreased, and maintain straight-line motion otherwise.
* Perpendicular Force Component: Causes the ball to change direction, momentarily bringing velocity to zero during turns.
* Velocity Graph Connection:
    - When the perpendicular force is small, the ball moves faster.
    - When it increases, the ball slows, halts, and reverses direction.
* Kinetic Energy Graph Connection:
    - The damping coefficient reduces kinetic energy over time by opposing motion.
    - Since velocity and kinetic energy are directly proportional ($KE∝v^2$), a reduction in velocity results in decreased kinetic energy.

**Overall Conclusion**:
We were able to identify clear relationships between forces, velocity, momentum, and kinetic energy. These relationships also allowed us to predict how changes in variables such as the damping coefficient would alter the system’s behavior.

## What We Learned

Through this lab, we learned how to:
1. Model Magnitude and Direction of 3D Components
    - Represent forces and motion vectors for a 3D object.
2. Add Components to Objects in VPython
    - Attached force vector attributes to the ball object.
3. Ensure Smooth Simulation Performance
    - Defined vector objects outside the while loop.
    - Updated force variables within the loop rather than re-creating vectors at each iteration.
4. Compute the Dot Product in VPython
    - Required for calculating the parallel force vector.
5. Integrate Kinetic Energy Properly
    - Converted kinetic energy from a vector form to a scalar value suitable for computation and graphing.
6. Understand Key Relationships Between Variables
    - Kinetic Energy and Velocity: Directly proportional.
    - Damping Coefficient: Resists motion, decreasing velocity and kinetic energy.
    - Parallel and Perpendicular Forces:
        - Parallel force regulates momentum and speed.
        - Perpendicular force changes direction, causing temporary halts in motion.
